In [1]:
import pandas as pd
from scipy.stats import chi2_contingency

In [2]:
pd.read_csv('result/subgroup_summary.csv')

,Subgroup,Count,Percentage
0,1,4407,0.755276
1,2,88557,15.176994
2,3,3296,0.564872
3,4,314891,53.966358
4,5,25966,4.450081
5,6,146378,25.086419


In [3]:
subgroup_1 = pd.read_csv('result/subgroup_1.csv')
subgroup_2 = pd.read_csv('result/subgroup_2.csv')
subgroup_3 = pd.read_csv('result/subgroup_3.csv')
subgroup_4 = pd.read_csv('result/subgroup_4.csv')
subgroup_5 = pd.read_csv('result/subgroup_5.csv')
subgroup_6 = pd.read_csv('result/subgroup_6.csv')

subgroups = [subgroup_1, subgroup_2, subgroup_3, subgroup_4, subgroup_5, subgroup_6]


In [91]:
subgroup_1['subgroup'] = 1

In [92]:
for index, i in enumerate(subgroups):
    i['subgroup'] = index + 1

In [93]:
concatenated_df = pd.concat(subgroups)
concatenated_df.head()

,subject_id,hadm_id,congestive_heart_failure,cardiac_arrhythmias,valvular_disease,pulmonary_circulation,peripheral_vascular,hypertension,paralysis,other_neurological,...,fluid_electrolyte,blood_loss_anemia,deficiency_anemias,alcohol_abuse,drug_abuse,psychoses,depression,age_group,admission_type,subgroup
0,6024,104644,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,4,0,1
1,6024,104644,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,4,0,1
2,6024,104644,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,4,0,1
3,6024,104644,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,4,0,1
4,6024,104644,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,4,0,1


__Rearragning the columns__

In [94]:
new_order = ['subgroup'] + [col for col in concatenated_df if col != 'subgroup']
concatenated_df = concatenated_df[new_order].drop(columns = ['subject_id','hadm_id'])

In [95]:
concatenated_df.head()

,subgroup,congestive_heart_failure,cardiac_arrhythmias,valvular_disease,pulmonary_circulation,peripheral_vascular,hypertension,paralysis,other_neurological,chronic_pulmonary,...,weight_loss,fluid_electrolyte,blood_loss_anemia,deficiency_anemias,alcohol_abuse,drug_abuse,psychoses,depression,age_group,admission_type
0,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,4,0
1,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,4,0
2,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,4,0
3,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,4,0
4,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,4,0


In [96]:
df = concatenated_df

__Only keep the categorical variable for chi-squared test__

In [97]:
results = {}

# Loop through all columns 
for column in df.columns:
    if column != 'subgroup':
        # Create the contingency table
        contingency = pd.crosstab(df['subgroup'], df[column])
        # Perform the chi-squared test
        chi2, p, dof, ex = chi2_contingency(contingency)
        # Store the results
        results[column] = {'chi2': chi2, 'p-value': p}


for variable, result in results.items():
    print(f"{variable}: chi2 statistic = {result['chi2']}, p-value = {result['p-value']}")

congestive_heart_failure: chi2 statistic = 748.8641137869972, p-value = 1.3316369194464204e-159
cardiac_arrhythmias: chi2 statistic = 15.3545866779349, p-value = 0.008949974005839911
valvular_disease: chi2 statistic = 181.75668605888202, p-value = 2.255484531425522e-37
pulmonary_circulation: chi2 statistic = 979.437714184468, p-value = 1.699872224622987e-209
peripheral_vascular: chi2 statistic = 5215.628459519085, p-value = 0.0
hypertension: chi2 statistic = 456563.7694555728, p-value = 0.0
paralysis: chi2 statistic = 3.847993788703312, p-value = 0.5715022574636488
other_neurological: chi2 statistic = 259.8517881975247, p-value = 4.2248640284683826e-54
chronic_pulmonary: chi2 statistic = 3981.8748522276255, p-value = 0.0
diabetes_uncomplicated: chi2 statistic = 8636.943310921219, p-value = 0.0
diabetes_complicated: chi2 statistic = 568617.3319185553, p-value = 0.0
hypothyroidism: chi2 statistic = 0.8530077402568188, p-value = 0.9735298716292372
renal_failure: chi2 statistic = 328637.58

__P value less than 0.05__

In [98]:
# Print the variables with a p-value less than 0.05
for variable, result in results.items():
    if result['p-value'] < 0.05:
        print(f"{variable}: chi2 statistic = {result['chi2']}, p-value = {result['p-value']}")

congestive_heart_failure: chi2 statistic = 748.8641137869972, p-value = 1.3316369194464204e-159
cardiac_arrhythmias: chi2 statistic = 15.3545866779349, p-value = 0.008949974005839911
valvular_disease: chi2 statistic = 181.75668605888202, p-value = 2.255484531425522e-37
pulmonary_circulation: chi2 statistic = 979.437714184468, p-value = 1.699872224622987e-209
peripheral_vascular: chi2 statistic = 5215.628459519085, p-value = 0.0
hypertension: chi2 statistic = 456563.7694555728, p-value = 0.0
other_neurological: chi2 statistic = 259.8517881975247, p-value = 4.2248640284683826e-54
chronic_pulmonary: chi2 statistic = 3981.8748522276255, p-value = 0.0
diabetes_uncomplicated: chi2 statistic = 8636.943310921219, p-value = 0.0
diabetes_complicated: chi2 statistic = 568617.3319185553, p-value = 0.0
renal_failure: chi2 statistic = 328637.58496569656, p-value = 0.0
liver_disease: chi2 statistic = 870.2921173403503, p-value = 7.149579204153824e-186
peptic_ulcer: chi2 statistic = 29.85701066700276,

In [106]:
contingency_table = pd.crosstab(df['subgroup'], df['age_group'])

contingency_table

age_group,1,2,3,4,5
subgroup,,,,,
1,0,4031,3,373,0
2,66099,0,0,0,22458
3,3282,0,14,0,0
4,26,3483,180577,128550,2255
5,9,24447,1510,0,0
6,2,146014,359,2,1


In [107]:
contingency_table = pd.crosstab(df['subgroup'], df['admission_type'])

contingency_table

admission_type,0,1
subgroup,,
1,4387,20
2,88225,332
3,3295,1
4,286653,28238
5,25908,58
6,146362,16


In [100]:
contingency_table = pd.crosstab(df['subgroup'], df['renal_failure'])

contingency_table

renal_failure,0,1
subgroup,,
1,2151,2256
2,88557,0
3,1106,2190
4,314731,160
5,25966,0
6,146378,0


In [101]:
contingency_table = pd.crosstab(df['subgroup'], df['blood_loss_anemia'])

contingency_table

blood_loss_anemia,0,1
subgroup,,
1,4407,0
2,88557,0
3,3296,0
4,314866,25
5,25966,0
6,146378,0


__P value greater than 0.05__

In [102]:
# Print the variables with a p-value less than 0.05
for variable, result in results.items():
    if result['p-value'] >= 0.05:
        print(f"{variable}: chi2 statistic = {result['chi2']}, p-value = {result['p-value']}")


paralysis: chi2 statistic = 3.847993788703312, p-value = 0.5715022574636488
hypothyroidism: chi2 statistic = 0.8530077402568188, p-value = 0.9735298716292372
weight_loss: chi2 statistic = 5.588929708905177, p-value = 0.3482929223295773
deficiency_anemias: chi2 statistic = 0.0, p-value = 1.0
alcohol_abuse: chi2 statistic = 4.558275113608532, p-value = 0.4721244919282802
drug_abuse: chi2 statistic = 0.0, p-value = 1.0
psychoses: chi2 statistic = 0.0, p-value = 1.0
depression: chi2 statistic = 0.0, p-value = 1.0


In [103]:
contingency_table = pd.crosstab(df['subgroup'], df['weight_loss'])

contingency_table

weight_loss,0,1
subgroup,,
1,4407,0
2,88556,1
3,3296,0
4,314891,0
5,25966,0
6,146378,0


In [105]:
contingency_table = pd.crosstab(df['subgroup'], df['depression'])

contingency_table

depression,0
subgroup,
1,4407
2,88557
3,3296
4,314891
5,25966
6,146378


In [3]:
import pandas as pd
from scipy.stats import chi2_contingency

# Load data
#subgroup_summary = pd.read_csv('subgroup_summary.csv')
subgroups = [pd.read_csv(f'result/subgroup_{i}.csv') for i in range(1, 7)]

# Add 'subgroup' column
for index, subgroup in enumerate(subgroups, start=1):
    subgroup['subgroup'] = index

# Concatenate and rearrange columns
concatenated_df = pd.concat(subgroups)
columns_order = ['subgroup'] + [col for col in concatenated_df if col != 'subgroup']
concatenated_df = concatenated_df[columns_order].drop(columns=['subject_id', 'hadm_id'])

# Perform chi-squared tests
results = {}
for column in concatenated_df.columns:
    if column != 'subgroup':
        contingency = pd.crosstab(concatenated_df['subgroup'], concatenated_df[column])
        chi2, p, _, _ = chi2_contingency(contingency)
        results[column] = {'chi2': chi2, 'p-value': p}

# Print results
for variable, result in results.items():
    significance = "Significant" if result['p-value'] < 0.05 else "Not Significant"
    print(f"{variable}: chi2 = {result['chi2']}, p-value = {result['p-value']} ({significance})")


congestive_heart_failure: chi2 = 748.8641137869972, p-value = 1.3316369194464204e-159 (Significant)
cardiac_arrhythmias: chi2 = 15.3545866779349, p-value = 0.008949974005839911 (Significant)
valvular_disease: chi2 = 181.75668605888202, p-value = 2.255484531425522e-37 (Significant)
pulmonary_circulation: chi2 = 979.437714184468, p-value = 1.699872224622987e-209 (Significant)
peripheral_vascular: chi2 = 5215.628459519085, p-value = 0.0 (Significant)
hypertension: chi2 = 456563.7694555728, p-value = 0.0 (Significant)
paralysis: chi2 = 3.847993788703312, p-value = 0.5715022574636488 (Not Significant)
other_neurological: chi2 = 259.8517881975247, p-value = 4.2248640284683826e-54 (Significant)
chronic_pulmonary: chi2 = 3981.8748522276255, p-value = 0.0 (Significant)
diabetes_uncomplicated: chi2 = 8636.943310921219, p-value = 0.0 (Significant)
diabetes_complicated: chi2 = 568617.3319185553, p-value = 0.0 (Significant)
hypothyroidism: chi2 = 0.8530077402568188, p-value = 0.9735298716292372 (No